In [6]:
import numpy as np 
import os
import glob
import cv2
import glob
import itertools
from skimage.morphology import skeletonize
import skimage.io as io
import skimage.transform as trans
import matplotlib.pyplot as plt
from math import atan2,degrees, atan
from mpl_toolkits.mplot3d import Axes3D
from skimage.morphology import skeletonize
from skimage.transform import hough_line, hough_line_peaks, probabilistic_hough_line
from matplotlib import cm
from shapely.geometry import LineString
from shapely.geometry import Point
from sympy import *
import math
from sympy.geometry import *
from skimage.measure import label, regionprops
from scipy.spatial import distance
from skimage.morphology import disk
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
#from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import precision_score, accuracy_score, roc_curve, auc

In [7]:
def Angular_Displacement(height, width, L1x1, L1y1, L1x2, L1y2, L2x1, L2y1, L2x2, L2y2):
    '''
    This code is for calculating Angular Displacement between a pair of lines.
    The predicted edge and mid line features can be evaluated quantitatively using the mean angular 
    displacement (mAD) error from the center of the image frame between the true andpredicted line.
    Angular displacement is calculated by the arc length distance between true point and predicted point.
    More details explanations of this metric see the paper please. 
    
    Input Arguments: 
        height = Height of the Image
        width  = Width of the Image
        L1x1 =  Initial X co-ordinate of the first line
        L1y1 =  Initial Y co-ordinate of the first line
        L1x2 =  Final X co-ordinate of the first line
        L1y2 =  Final Y co-ordinate of the first line
        L2x1 =  Initial X co-ordinate of the second line
        L2y1 =  Initial Y co-ordinate of the second line
        L2x2 =  Final X co-ordinate of the second line
        L2y2 =  Final Y co-ordinate of the second line
        
    Output Arguments:  
    
        AngularDisplacement: The average Angular Displacement of left and right side Displacement.
        
    Author: Md. Kamrul Hasan, 
        Erasmus Scholar on Medical Imaging and Application (MAIA)
        E-mail: kamruleeekuet@gmail.com
        
    '''

    center_x = int(height/2)
    center_y = int(width/2)
    centerPoint = Point(center_x,center_y)
    radius = np.sqrt((center_x)**2+(center_y)**2)

    circle = Circle(centerPoint, radius)

    lineApprox = Line((L1x1,L1y1),(L1x2,L1y2))
    lineTrue = Line((L2x1,L2y1),(L2x2,L2y2))

    intersectionApprox= intersection(circle, lineApprox)
    intersectionTrue= intersection(circle, lineTrue)

    intXY1P = intersectionApprox[0].evalf()
    X_P_1 = intXY1P.x
    Y_P_1 = intXY1P.y
#         print(X_P_1)
#         print(Y_P_1)

    intXY2P = intersectionApprox[1].evalf()
    X_P_2 = intXY2P.x
    Y_P_2 = intXY2P.y
#         print(X_P_2)
#         print(Y_P_2)


    intXY1G = intersectionTrue[0].evalf()
    X_G_1 = intXY1G.x
    Y_G_1 = intXY1G.y
#         print(X_G_1)
#         print(Y_G_1)


    intXY2G = intersectionTrue[1].evalf()
    X_G_2 = intXY2G.x
    Y_G_2 = intXY2G.y
#         print(X_G_2)
#         print(Y_G_2)

    slope1_P, intercept=slope_intercept(X_P_1, Y_P_1, center_x, center_y)
    slope1_G, intercept=slope_intercept(X_G_1, Y_G_1, center_x, center_y)

    slope2_P, intercept=slope_intercept(X_P_2, Y_P_2, center_x, center_y)
    slope2_G, intercept=slope_intercept(X_G_2, Y_G_2, center_x, center_y)

#         print(slope1_P)
#         print(slope1_G)
#         print(slope2_P)
#         print(slope2_G)

    theta_1=atan(abs((slope1_P-slope1_G)/(1+slope1_G*slope1_P)))
    theta_2=atan(abs((slope2_P-slope2_G)/(1+slope2_G*slope2_P)))
    
    AngularDisplacement=(theta_1+theta_2)/2
    
    return AngularDisplacement



def Performance_Metrics(y_true, y_pred):
    ''' 
    The Dice similarity coefficient (DSC) is a statistical validation metric for the image
    segmentations and determines the spatial overlap accuracy of the segmentation. It also 
    same as F1-score. 
    
    The Intersection over Union (IoU) also referred to as the Jaccard index (JI), is essentially
    a method to quantify the percent overlap between the GT mask and prediction output.
    This metric is closely related to the DSC. The IoU metric measures the number of pixels
    common between the target and prediction masks divided by the total number of pixels present
    across both masks.
    
    Mathematically, they are related as follows:
    IoU = DSC/(2-DSC) or DSC = 2*IoU/(IoU+1)
    
    The Sensitivity also called the true positive rate, the recall, or probability of detection that 
    measures the proportion of actual positives that are correctly identified as such class 
    (e.g., the percentage of sick people who are correctly identified as having the condition).
    
    The Specificity also called the true negative rate measures the proportion of actual negatives that
    are correctly identified as such class. (e.g., the percentage of healthy people who are correctly
    identified as not having the condition).
    
  
    Input Arguments: 
        y_true: True Labels of the 2D images so called ground truth (GT).
        y_pred: Predicted Labels of the 2D images so called Predicted/ segmented Mask.
        
    Output Arguments: 
        dsc: The DSC between y_true and y_pred
        iou: The IoU between y_true and y_pred
        accuracy: The accuracy of pixels clasification.
        sensitivity: The accuracy of foreground pixels clasification.
        specificity: The accuracy of background pixels clasification.
        balancedAccuracy: The balanced accuracy of both foreground & background pixels clasification.
        
    Author: Md. Kamrul Hasan, 
            Erasmus Scholar on Medical Imaging and Application (MAIA)
            E-mail: kamruleeekuet@gmail.com

    '''
    if y_true.shape != y_pred.shape:
        raise ValueError("Shape mismatch!! y_true and y_pred must have the same shape.")

    y_true_f = (y_true/y_true.max()).flatten()
    y_pred_f = (y_pred/y_pred.max()).flatten()
    
    intersection = np.sum(y_true_f * y_pred_f)
    
    dsc = (2. * intersection ) / (np.sum(y_true_f) + np.sum(y_pred_f))
    iou = (intersection) / (np.sum(y_true_f) + np.sum(y_pred_f)-intersection)
    
    y_true = np.asarray(y_true).astype(np.bool)
    y_pred = np.asarray(y_pred).astype(np.bool)

        
    y_true=y_true.flatten()
    y_pred=y_pred.flatten()
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (tn+tp)/(tn+fp+fn+tp)
    sensitivity = (tp)/(fn+tp)
    specificity = (tn)/(tn+fp)
    balancedAccuracy = (sensitivity+specificity)/2

    
    return dsc, iou, accuracy, sensitivity, specificity, balancedAccuracy



def slope_intercept(xi, yi, xf, yf):
    '''
    This function is dedicated for calculating the slope of the line. 
    
    Input Arguments: 
         xi: The initial X co-ordinate
         yi: The initial Y co-ordinate
         xf: The final X co-ordinate
         yf: The final Y co-ordinate
         
    Output Arguments:
        slope: Slope of the st. line connecting those two points
        intercept: The intersecting length from Y-axis
         
    Author: Md. Kamrul Hasan, 
        Erasmus Scholar on Medical Imaging and Application (MAIA)
        E-mail: kamruleeekuet@gmail.com
        
    '''
    
    slope = (yf - yi) / (xf - xi)
    intercept = yi - slope * xi 
    
    return slope,intercept

## Evaluate Segmentation 

In [8]:
CurrentDirectory=os.getcwd()
PredictionPath = '/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/'
Prediction = glob.glob(PredictionPath+"*.png")
Prediction.sort()
# print(len(Prediction))

OriginalmaskGT = glob.glob('/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Test_Positive_Tool_Mask/'+"*.png")
OriginalmaskGT.sort()
# print(len(OriginalmaskGT))

OriginalImage = glob.glob('/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Test_Positive/'+"*.png")
OriginalImage.sort()
# print(len(OriginalImage))

masksavePath = '/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Results_Evaluation/Tool_Mask/'

i=0

dice=[]
sensitivity=[]
specificity=[]
accuracy = []
BalancedAccuracy=[]
IoU=[]

for Pred in Prediction:
    
    filename, file_extension = os.path.splitext(Pred) 
    if 'tool_mask' in filename:
        print(filename)
        #----------------------------------Read the Prediction from SIMO------------------------------
        imgPred = cv2.imread(Pred,0)
        
        # Apply the threshold since output was sigmoid activations of segmentation sub-network
        threshold = 30
        imgPred[imgPred<threshold] = 0
        imgPred[imgPred>=threshold] = 255
        
        #--------------------------------Read Original Image with Mask GT----------------------------
        orimg = cv2.imread(OriginalImage[i],-1)
        orimg = cv2.resize(orimg,(256,192))
        
        truemask = cv2.imread(OriginalmaskGT[i],0)
        truemask = cv2.resize(truemask,(256,192))
        i=i+1
        
        #-------------------------------Overlay with original Image------------------------------------       
        temp = cv2.merge((imgPred, imgPred, imgPred))
        temp [:,:,0]=0
        temp [:,:,2]=0       

        overlay = cv2.addWeighted(temp, 0.5, orimg, 0.8, 0)
        
     
        # Call the Performance Evaluation functions by passing predicted and true mask
        dsc, iou, acc, sen, spe, bAcc = Performance_Metrics(imgPred, truemask)

        dice.append(dsc)
        IoU.append(iou)
        accuracy.append(acc)
        sensitivity.append(sen)
        specificity.append(spe)
        BalancedAccuracy.append(bAcc)
        
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (80,185)
        fontScale              = 1
        fontColor              = (255,255,255)
        lineType               = 2
        

        cv2.putText(overlay, str(round(dsc,4)), 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineType)
        
        savelinePath=masksavePath+'ImgID_'+filename[-14:]+'_Overlay_with_OrgImg.png'
        cv2.imwrite(savelinePath, overlay)  
        print('ImageID: '+ filename[-12]+'. Dice co-efficient: '+ str(dsc)
              + '  Balanced Accuracy: ' + str(bAcc))
        
print()
print('-----------------------Final Segmentation metrics are as follows:---------------------------')
print("DSC = %0.4f (+/-%0.04f)" % (np.mean(dice)*100, np.std(dice)*100))
print("Sensitivity = %0.4f (+/-%0.04f)" % (np.mean(sensitivity)*100, np.std(sensitivity)*100))
print("Specificity = %0.4f (+/-%0.04f)" % (np.mean(specificity)*100, np.std(specificity)*100))
print("Accuracy = %0.8f (+/-%0.04f)" % (np.mean(accuracy)*100, np.std(accuracy)*100))
print("IoU = %0.4f (+/-%0.04f)" % (np.mean(IoU)*100, np.std(IoU)*100))
print("Balanced Accuracy = %0.4f (+/-%0.04f)" % (np.mean(BalancedAccuracy)*100, np.std(BalancedAccuracy)*100))

/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0001_tool_mask
ImageID: 0. Dice co-efficient: 0.9097617672376123  Balanced Accuracy: 0.9463016270628422
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0002_tool_mask
ImageID: 0. Dice co-efficient: 0.926805203647147  Balanced Accuracy: 0.9402082292098479
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0003_tool_mask
ImageID: 0. Dice co-efficient: 0.9714187742069672  Balanced Accuracy: 0.9849306379234262
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0004_tool_mask
ImageID: 0. Dice co-efficient: 0.9807261960178959  Balanced Accuracy: 0.9858891866051623
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0005_tool_mask
ImageID: 0. Dice co-efficient: 0.9756490342109849  Balanced Accuracy: 0.9820466701708994
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentati

ImageID: 4. Dice co-efficient: 0.9723480968797349  Balanced Accuracy: 0.9842082400236197
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0049_tool_mask
ImageID: 4. Dice co-efficient: 0.9521963789360504  Balanced Accuracy: 0.9761266333964602
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0050_tool_mask
ImageID: 5. Dice co-efficient: 0.9580682611052308  Balanced Accuracy: 0.9717191820256255
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0051_tool_mask
ImageID: 5. Dice co-efficient: 0.9556530396624612  Balanced Accuracy: 0.9668012283344527
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0052_tool_mask
ImageID: 5. Dice co-efficient: 0.91577001828249  Balanced Accuracy: 0.9582130067512743
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0053_tool_mask
ImageID: 5. Dice co-efficient: 0.9714228171557083  B

ImageID: 0. Dice co-efficient: 0.9364614951804198  Balanced Accuracy: 0.9713450806518074
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0103_tool_mask
ImageID: 0. Dice co-efficient: 0.966742747276727  Balanced Accuracy: 0.9799966425740696
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0104_tool_mask
ImageID: 0. Dice co-efficient: 0.9040641623507948  Balanced Accuracy: 0.9252865663964891
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0105_tool_mask
ImageID: 0. Dice co-efficient: 0.8668899733979364  Balanced Accuracy: 0.9006085119146151
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0106_tool_mask
ImageID: 0. Dice co-efficient: 0.8859880274036996  Balanced Accuracy: 0.907278977794119
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0107_tool_mask
ImageID: 0. Dice co-efficient: 0.9609899777792535  B

ImageID: 5. Dice co-efficient: 0.9308922911925862  Balanced Accuracy: 0.9579191939582112
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0151_tool_mask
ImageID: 5. Dice co-efficient: 0.9365145153442777  Balanced Accuracy: 0.9723054971933895
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0152_tool_mask
ImageID: 5. Dice co-efficient: 0.9408945958642313  Balanced Accuracy: 0.9570625415001229
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0153_tool_mask
ImageID: 5. Dice co-efficient: 0.9367885003601245  Balanced Accuracy: 0.9511193863050682
/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/Test_Pos_sample_0154_tool_mask
ImageID: 5. Dice co-efficient: 0.9696264841125272  Balanced Accuracy: 0.9866760235361995

-----------------------Final Segmentation metrics are as follows:---------------------------
DSC = 92.9240 (+/-6.3990)
Sensitivity = 92.9334 (+/-5.550

## Mid line mean angular displacement performance

In [12]:
CurrentDirectory=os.getcwd()
PredictionPath = '/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/'
Prediction = glob.glob(PredictionPath+"*.png")
Prediction.sort()
# print(len(Prediction))

OriginalMidlineGT = glob.glob('/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Test_Positive_MidLine/'+"*.png")
OriginalMidlineGT.sort()
# print(len(OriginalmaskGT))

OriginalImage = glob.glob('/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Test_Positive/'+"*.png")
OriginalImage.sort()
# print(len(OriginalImage))

midlinesavePath = '/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Results_Evaluation/Mid_line/'


i=0


AD_midline=[]
for Pred in Prediction:
    
    filename, file_extension = os.path.splitext(Pred) 
    
    if 'midline' in filename:
        #------------------------------------Read the Prediction from SIMO--------------------------------
        imgPred = cv2.imread(Pred,0)
        tempimgPred = imgPred.copy()
        #-----------------------------------Read Original Image with Midline GT---------------------------
        orimg = cv2.imread(OriginalImage[i],-1)
        orimg = cv2.resize(orimg,(256,192))
        
        trueLine = cv2.imread(OriginalMidlineGT[i],0)
        trueLine = cv2.resize(trueLine,(256,192))
        i=i+1
        #-------------------------------Truncate the intensity lower than Threshold----------------------
        Threshold = (imgPred.max())-100
        imgPred[imgPred<Threshold]=0
        
        temp = cv2.merge((tempimgPred, tempimgPred, tempimgPred))
        temp [:,:,0]=0
        temp [:,:,2]=0 

        
        savelinePath=midlinesavePath+'ImgID_'+filename[-11:]+'_Prediction_.png'
        cv2.imwrite(savelinePath,cv2.addWeighted(temp,0.99,orimg,0.8,0))
                
        
        #-----------------------------Classic straight-line Hough transform----------------------------
        h, theta, d = hough_line(imgPred)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d, min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0P = 0
        y0P = int(y0)
        x1P = imgPred.shape[1]
        y1P = int(y1)
        
        #-----------------------------Approximate Line-------------------------------------------------
        ApproximateLine=np.zeros_like(imgPred, dtype=np.uint8)
    
        cv2.line(ApproximateLine,(x0P,y0P),(x1P,y1P), 255, 1)
        
#         savelinePath=midlinesavePath+filename[-11:]+'_Approx_.png'
#         cv2.imwrite(savelinePath,ApproximateLine)  
        
        #---------------------------True Line approximation----------------------------------------------
        h, theta, d = hough_line(trueLine)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0G = 0
        y0G = int(y0)
        x1G= imgPred.shape[1]
        y1G= int(y1)
        
        trueLine=np.zeros_like(trueLine, dtype=np.uint8)
    
        cv2.line(trueLine,(x0G,y0G),(x1G,y1G), 255, 1)   
        
        
        #-----------------------Overlay with original Image------------------------------------------       
        temp = cv2.merge((ApproximateLine, ApproximateLine, ApproximateLine))
        temp [:,:,0]=0
        temp [:,:,2]=0 
        
        temp2 = cv2.merge((trueLine, trueLine, trueLine))
        temp2 [:,:,0]=0 

        
        dummy = cv2.addWeighted(temp,0.99,orimg,0.8,0)
        
        dummy = cv2.addWeighted(temp2,0.99,dummy,0.8,0)
        
        
        #----------------------mean Angular Displacement Error Measurements--------------------------

        AngularDisplacement=Angular_Displacement(height = 192,
                                                width = 256,
                                                L1x1 = x0P,
                                                L1y1 = y0P,
                                                L1x2 = x1P,
                                                L1y2 = y1P,
                                                L2x1 = x0G,
                                                L2y1 = y0G,
                                                L2x2 = x1G,
                                                L2y2 = y1G)
        
        print('ImageID: '+ filename[-11:-8]+'.  AD in radian: '+ str(AngularDisplacement)
              + '  AD in degrees: ' + str(degrees(AngularDisplacement)))
        
        
        #------------------------ApproximateLine with True Line---------------------------------------
        
        TruePlusApproximation = ApproximateLine+trueLine
        
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (80,185)
        fontScale              = 1
        fontColor              = (255,255,255)
        lineType               = 2

        cv2.putText(dummy,str(round (degrees(AngularDisplacement),2)), 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineType)

#         savelinePath=midlinesavePath+filename[-11:]+'_ApproxPlusTrue_.png'
#         cv2.imwrite(savelinePath,TruePlusApproximation) 
        
        savelinePath=midlinesavePath+'ImgID_'+filename[-11:]+'_Overlay_with_Approx_true_.png'
        cv2.imwrite(savelinePath,dummy)  

        AD_midline.append(AngularDisplacement)
  
print()
print('-----------------------Final mean Angular Displacement (mAD)---------------------------')
print()
print('mAD in radians: '+ str(np.mean(AD_midline)) + '  mAD in degrees: ' + str(degrees(np.mean(AD_midline))))
print('std. AD in radians: '+ str(math.sqrt(np.mean((np.array(AD_midline)-np.mean(AD_midline))**2))) 
      + '  median AD in radians: ' + str(np.median(AD_midline)))

ImageID: 001.  AD in radian: 0.0193529796777810  AD in degrees: 1.1088440565393032
ImageID: 002.  AD in radian: 0.106437890272833  AD in degrees: 6.098441892909874
ImageID: 003.  AD in radian: 0.0572832193014515  AD in degrees: 3.282086702895505
ImageID: 004.  AD in radian: 0.298601844362136  AD in degrees: 17.108625436772655
ImageID: 005.  AD in radian: 0.313497999634434  AD in degrees: 17.962112264846887
ImageID: 006.  AD in radian: 0.0202261334223564  AD in degrees: 1.1588720809695152
ImageID: 007.  AD in radian: 0.0367613415983628  AD in degrees: 2.1062697228248966
ImageID: 008.  AD in radian: 0.0179297698816290  AD in degrees: 1.0273001418581185
ImageID: 009.  AD in radian: 0.124830050311230  AD in degrees: 7.152235039239212
ImageID: 010.  AD in radian: 0.0855515063749158  AD in degrees: 4.901740246269234
ImageID: 011.  AD in radian: 0.0181301593709445  AD in degrees: 1.0387816138546797
ImageID: 012.  AD in radian: 0.0196276873981894  AD in degrees: 1.124583649518362
ImageID: 013.

ImageID: 105.  AD in radian: 0.0520861735579673  AD in degrees: 2.984317915857435
ImageID: 106.  AD in radian: 0.305463723886031  AD in degrees: 17.501782173019084
ImageID: 107.  AD in radian: 0.228815201484699  AD in degrees: 13.110145333508813
ImageID: 108.  AD in radian: 0.0528735604390387  AD in degrees: 3.029431860986791
ImageID: 109.  AD in radian: 0.0857178068956618  AD in degrees: 4.911268564238807
ImageID: 110.  AD in radian: 0.847556029825817  AD in degrees: 48.561383409883454
ImageID: 111.  AD in radian: 0.0878251937417089  AD in degrees: 5.032012936318691
ImageID: 112.  AD in radian: 0.350706148009596  AD in degrees: 20.09398213024024
ImageID: 113.  AD in radian: 1.17447394216789  AD in degrees: 67.2924000343122
ImageID: 114.  AD in radian: 0.0134469445953261  AD in degrees: 0.7704531726584404
ImageID: 115.  AD in radian: 0.123033463097079  AD in degrees: 7.049298174341198
ImageID: 116.  AD in radian: 0.0179680225715584  AD in degrees: 1.0294918595460947
ImageID: 117.  AD i

## Edge lines mean angular displacement performance

In [13]:
testGene = '/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/'
Prediction = glob.glob(testGene+"*.png")
Prediction.sort()
#print(len(Prediction))

OriginalupGT = glob.glob('/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Test_Positive_EdgeLine_1/'+"*.png")
OriginalupGT.sort()
#print(len(OriginalupGT))


OriginallowGT = glob.glob('/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Test_Positive_EdgeLine_2/'+"*.png")
OriginallowGT.sort()
#print(len(OriginallowGT))


OriginalImage = glob.glob('/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Test_Positive/'+"*.png")
OriginalImage.sort()
#print(len(OriginalImage))

midlinesavePath = '/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Results_Evaluation/EdgeLines/'


i=0

AngularDisplacement_Line_1 = []
AngularDisplacement_Line_2 = []
bothError=[]

for Pred in Prediction:
    
    filename, file_extension = os.path.splitext(Pred) 
    
    if 'edgeLine' in filename:
        #--------------------------Read the Prediction from SIMO----------------------------------------
        imgPred = cv2.imread(Pred,0)
        temp = imgPred.copy()
        temp2 = imgPred.copy()
        
        #-------------------------Read Original Image with Midline GT-----------------------------------
        orimg = cv2.imread(OriginalImage[i],-1)
        orimg = cv2.resize(orimg,(256,192))
        
        up = cv2.imread(OriginalupGT[i],0)
        up = cv2.resize(up,(256,192))
        
        up[up<100]=0
        up[up>100]=255
        
        low = cv2.imread(OriginallowGT[i],0)
        low = cv2.resize(low,(256,192))
        low[low<100]=0
        low[low>100]=255
        
        trueline=up+low
        
        i=i+1

        #-----------------------Truncate the intensity lower than Threshold--------------------------
        imgPredtemp=imgPred.copy()
        
        Threshold = (imgPred.max())-100
        imgPred[imgPred<Threshold]=0
        imgPredtemp[imgPredtemp<Threshold-30]=0
        
        
        
#         savelinePath=midlinesavePath+filename[-13:]+'_Pred_.png'
#         cv2.imwrite(savelinePath,imgPred)
        
#         savelinePath=midlinesavePath+filename[-13:]+'_true_.png'
#         cv2.imwrite(savelinePath,trueline)
        
#         savelinePath=midlinesavePath+filename[-13:]+'_truePLUSPred_.png'
#         cv2.imwrite(savelinePath,trueline+imgPred)
                
        
        #-----------------------Classic straight-line Hough transform-------------------------------
        h, theta, d = hough_line(imgPred)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0P1 = 0
        y0P1 = int(y0)
        x1P1 = imgPred.shape[1]
        y1P1 = int(y1)
        
        #-------------------------Approximate Line_1------------------------------------------------
        ApproximateLine1=np.zeros_like(imgPred, dtype=np.uint8)
    
        cv2.line(ApproximateLine1,(x0P1,y0P1),(x1P1,y1P1), 255, 1)       
        
        #------------------------Classic straight-line Hough transform------------------------------
        temp=np.zeros_like(imgPred, dtype=np.uint8)
        temp=cv2.line(temp,(x0P1,y0P1),(x1P1,y1P1), 255, 15)
        
        temp = 255-temp
        
        imgPred=imgPred & temp
        
        h, theta, d = hough_line(imgPred)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0P2 = 0
        y0P2 = int(y0)
        x1P2 = imgPred.shape[1]
        y1P2 = int(y1)
        
        #-------------------------Approximate Line_2------------------------------------------------
        ApproximateLine2=np.zeros_like(imgPred, dtype=np.uint8)
    
        cv2.line(ApproximateLine2,(x0P2,y0P2),(x1P2,y1P2), 255, 1)
        
        ApproximateLine = ApproximateLine2 + ApproximateLine1
        

        cv2.imwrite(midlinesavePath+filename[-13:]+'_Line_1_.png',ApproximateLine1) 
        cv2.imwrite(midlinesavePath+filename[-13:]+'_Line_2_.png',ApproximateLine2) 
        
        
        #-----------------------True Line-1 approximation------------------------------------------
        h, theta, d = hough_line(trueline)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0G1 = 0
        y0G1 = int(y0)
        x1G1= imgPred.shape[1]
        y1G1= int(y1)
        
        up=np.zeros_like(trueline, dtype=np.uint8)
    
        cv2.line(up,(x0G1,y0G1),(x1G1,y1G1), 255, 1)  
        
        temp=np.zeros_like(trueline, dtype=np.uint8)
    
        cv2.line(temp,(x0G1,y0G1),(x1G1,y1G1), 255, 15) 
        temp = 255-temp
        
        trueline= trueline & temp
        
        labelImage=label(trueline)
        regions=regionprops(labelImage)
        Area=[]
        for region in range(len(regions)):
            maxThreshold=regions[region].area,
            Area.append(maxThreshold)
        X= sorted( [(x,i) for (i,x) in enumerate(Area)], reverse=True )[:2]

        index=np.array(X)
        V=index.T[1].tolist()
        trueline[labelImage!=(V[0]+1)]=0
        
        h, theta, d = hough_line(trueline)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0G2 = 0
        y0G2 = int(y0)
        x1G2= imgPred.shape[1]
        y1G2= int(y1)
        
        low=np.zeros_like(trueline, dtype=np.uint8)
    
        cv2.line(low,(x0G2,y0G2),(x1G2,y1G2), 255, 1)   
        
        trueline = up+low

#         savelinePath=midlinesavePath+filename[-13:]+'_ApproxPLUStrue_.png'
#         cv2.imwrite(savelinePath,ApproximateLine+trueline) 
        
        #-------------------Overlay with original Image-------------------------------       
        temp = cv2.merge((ApproximateLine, ApproximateLine, ApproximateLine))
        temp [:,:,0]=0
        temp [:,:,2]=0  
        
        temp1 = cv2.merge((trueline, trueline, trueline))
        temp1 [:,:,0]=0

        
        dummy1 = cv2.addWeighted(temp,0.99,orimg,0.8,0)
        dummy2 = cv2.addWeighted(temp1,0.99,dummy1,0.8,0)
        
#         savelinePath=midlinesavePath+filename[-13:]+'_overlay_.png'
#         cv2.imwrite(savelinePath,dummy2) 
        
        
        AngularDisplacement_approx1_up=Angular_Displacement(height = 192,
                                                            width = 256,
                                                            L1x1 = x0P1,
                                                            L1y1 = y0P1,
                                                            L1x2 = x1P1,
                                                            L1y2 = y1P1,
                                                            L2x1 = x0G1,
                                                            L2y1 = y0G1,
                                                            L2x2 = x1G1,
                                                            L2y2 = y1G1)
        
        AngularDisplacement_approx1_low=Angular_Displacement(height = 192,
                                                            width = 256,
                                                            L1x1 = x0P1,
                                                            L1y1 = y0P1,
                                                            L1x2 = x1P1,
                                                            L1y2 = y1P1,
                                                            L2x1 = x0G2,
                                                            L2y1 = y0G2,
                                                            L2x2 = x1G2,
                                                            L2y2 = y1G2)
        
        AngularDisplacement_approx2_up=Angular_Displacement(height = 192,
                                                            width = 256,
                                                            L1x1 = x0P2,
                                                            L1y1 = y0P2,
                                                            L1x2 = x1P2,
                                                            L1y2 = y1P2,
                                                            L2x1 = x0G1,
                                                            L2y1 = y0G1,
                                                            L2x2 = x1G1,
                                                            L2y2 = y1G1)
        
        AngularDisplacement_approx2_low=Angular_Displacement(height = 192,
                                                            width = 256,
                                                            L1x1 = x0P2,
                                                            L1y1 = y0P2,
                                                            L1x2 = x1P2,
                                                            L1y2 = y1P2,
                                                            L2x1 = x0G2,
                                                            L2y1 = y0G2,
                                                            L2x2 = x1G2,
                                                            L2y2 = y1G2)
        
        if AngularDisplacement_approx1_up>AngularDisplacement_approx1_low:
            line_1_Error = AngularDisplacement_approx1_low
        else:
            line_1_Error = AngularDisplacement_approx1_up
    
        AngularDisplacement_Line_1.append(line_1_Error)
        
        if AngularDisplacement_approx2_up>AngularDisplacement_approx2_low:
            line_2_Error = AngularDisplacement_approx2_low
        else:
            line_2_Error = AngularDisplacement_approx2_up
    
        AngularDisplacement_Line_2.append(line_2_Error)

        Error= (line_1_Error+line_2_Error)/2
        
        bothError.append(Error)
        
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (80,185)
        fontScale              = 1
        fontColor              = (255,255,255)
        lineType               = 2

        cv2.putText(dummy2,str(round(degrees(Error),2)), 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineType)
        savelinePath=midlinesavePath+'ImgID_'+filename[-12:]+'_Pred_True-EdgeLine.png'
        cv2.imwrite(savelinePath,dummy2) 
        
        print('Angular Displacement (AD) in radian: '+ str(Error)
              + '  Angular Displacement (AD) in degrees: ' + str(degrees(Error)))

        
  
print()
print('-----------------------Final mean Angular Displacement (mAD)---------------------------')
print()
print('mAD in radians: '+ str(np.mean(bothError)) + '  mAD in degrees: ' + str(degrees(np.mean(bothError))))
print('std. AD in radians: '+ str(math.sqrt(np.mean((np.array(bothError)-np.mean(bothError))**2))) 
      + '  median AD in radians: ' + str(np.median(bothError)))

<ipython-input-13-07d3d34f27d0>:150: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  index=np.array(X)


Angular Displacement (AD) in radian: 0.0111387166358405  Angular Displacement (AD) in degrees: 0.6382014524258175
Angular Displacement (AD) in radian: 0.0207124699565603  Angular Displacement (AD) in degrees: 1.1867371118024208
Angular Displacement (AD) in radian: 0.0169980228730903  Angular Displacement (AD) in degrees: 0.9739149706949138
Angular Displacement (AD) in radian: 0.0280428152606077  Angular Displacement (AD) in degrees: 1.6067349600978782
Angular Displacement (AD) in radian: 0.0325537540977238  Angular Displacement (AD) in degrees: 1.8651927171062834
Angular Displacement (AD) in radian: 0.0228598872339843  Angular Displacement (AD) in degrees: 1.309775058652288
Angular Displacement (AD) in radian: 0.0183071310004733  Angular Displacement (AD) in degrees: 1.04892134132023
Angular Displacement (AD) in radian: 0.0260922089699861  Angular Displacement (AD) in degrees: 1.4949734521535938
Angular Displacement (AD) in radian: 0.00974565121828136  Angular Displacement (AD) in degr

Angular Displacement (AD) in radian: 0.0242375957541919  Angular Displacement (AD) in degrees: 1.388711942259398
Angular Displacement (AD) in radian: 0.00877636965730678  Angular Displacement (AD) in degrees: 0.5028489408103554
Angular Displacement (AD) in radian: 0.0111160374250142  Angular Displacement (AD) in degrees: 0.6369020293627822
Angular Displacement (AD) in radian: 0.0188351203965697  Angular Displacement (AD) in degrees: 1.0791729053442158
Angular Displacement (AD) in radian: 0.0351892857112322  Angular Displacement (AD) in degrees: 2.016197555333618
Angular Displacement (AD) in radian: 0.0180466208469011  Angular Displacement (AD) in degrees: 1.033995209000239
Angular Displacement (AD) in radian: 0.0197964222316717  Angular Displacement (AD) in degrees: 1.1342514433337456
Angular Displacement (AD) in radian: 0.0268232792917162  Angular Displacement (AD) in degrees: 1.5368606961159976
Angular Displacement (AD) in radian: 0.580595359952476  Angular Displacement (AD) in degre

Angular Displacement (AD) in radian: 0.00891049913077775  Angular Displacement (AD) in degrees: 0.5105339935485536
Angular Displacement (AD) in radian: 0.0449534541738882  Angular Displacement (AD) in degrees: 2.5756431986985477
Angular Displacement (AD) in radian: 0.0791356275440221  Angular Displacement (AD) in degrees: 4.534137467391693
Angular Displacement (AD) in radian: 0.0119831060809221  Angular Displacement (AD) in degrees: 0.6865814038943906
Angular Displacement (AD) in radian: 0.00782975855484339  Angular Displacement (AD) in degrees: 0.4486121197989772
Angular Displacement (AD) in radian: 0.199195774819662  Angular Displacement (AD) in degrees: 11.413077194004979
Angular Displacement (AD) in radian: 0.0203332486022351  Angular Displacement (AD) in degrees: 1.1650093286983496
Angular Displacement (AD) in radian: 0  Angular Displacement (AD) in degrees: 0.0

-----------------------Final mean Angular Displacement (mAD)---------------------------

mAD in radians: 0.040523901933

## Tool tip Evaluation Eculedian Distance 

In [11]:
PredictionPath = '/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/Jupyter/Results/'
Prediction = glob.glob(PredictionPath+"*.png")
Prediction.sort()
# print(len(Prediction))

OriginaltipGT = glob.glob('/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Test_Positive_TipPoint/'+"*.png")
OriginaltipGT.sort()
# print(len(OriginaltipGT))

OriginalImage = glob.glob('/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Test_Positive/'+"*.png")
OriginalImage.sort()
# print(len(OriginalImage))

tipsavePath = '/home/mahmoud/Desktop/laparoscopic-Tools-Segmentation/DataFinal/Test/Results_Evaluation/Tip/'

i=0

Euclidean_Tip=[]

for Pred in Prediction:
    
    filename, file_extension = os.path.splitext(Pred) 

    if 'tip' in filename:
        #--------------------------Read the Prediction from SIMO----------------------------------
        imgPred = cv2.imread(Pred,0)
        temp = imgPred.copy()
        
        #-------------------------Read Original Image with TIP GT---------------------------------
        orimg = cv2.imread(OriginalImage[i],-1)
        orimg = cv2.resize(orimg,(256,192))
        
        truepoint = cv2.imread(OriginaltipGT[i],0)
        truepoint = cv2.resize(truepoint,(256,192))
        i=i+1

        #----------------------Truncate the intensity lower than Threshold-----------------------
        Threshold = (imgPred.max())-100
        imgPred[imgPred<Threshold]=0
               
#         savelinePath=tipsavePath+filename[-7:]+'_Pred_.png'
#         cv2.imwrite(savelinePath,imgPred)
        
        #-----------------------Overlay with original Image--------------------------------------       
        temp = cv2.merge((imgPred, imgPred, imgPred))
        temp [:,:,0]=0
        temp [:,:,2]=0   
        
        temp2 = cv2.merge((truepoint, truepoint, truepoint))
        temp2 [:,:,0]=0

        dummy = cv2.addWeighted(temp, 0.8, orimg, 0.8, 0)
        dummy = cv2.addWeighted(temp2, 0.8, dummy, 0.8, 0)
        
                
        #------------------------Euclidean Distance-----------------------------------------------
        XY_PredictedPoint = np.argwhere(imgPred.max() == imgPred)
        XY_truePoint = np.argwhere(truepoint.max() == truepoint)
        
        X_P = XY_PredictedPoint[0][1]
        Y_P = XY_PredictedPoint[0][0]
        
        X_G = XY_truePoint[0][1]
        Y_G = XY_truePoint[0][0]  

        euclidean_distance = distance.euclidean((X_P, Y_P), (X_G, Y_G))
        print('ImageID: '+ filename[-7:-4]+ '.   Euclidean Distance in pixels: '+ str(euclidean_distance))
        Euclidean_Tip.append(euclidean_distance)
        
        #--------------------Approximate point with True point---------------------------------------
        
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (80,185)
        fontScale              = 1
        fontColor              = (255,255,255)
        lineType               = 2

        cv2.putText(dummy,str(round(euclidean_distance,2)), 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineType)

        savelinePath=tipsavePath+'ImgID_'+filename[-12:]+'_Overlay_Pred_True.png'
        cv2.imwrite(savelinePath,dummy)  

        
print()
print('-----------------------Final Mean Euclidean Distance (mED)---------------------------')
print()
print('Mean Euclidean Distance (mED) in pixels: '+ str(np.mean(Euclidean_Tip)))
print('std. Euclidean Distance (ED): '+ str(math.sqrt(np.mean((np.array(Euclidean_Tip)-np.mean(Euclidean_Tip))**2))) 
      + '  median Euclidean Distance (ED): ' + str(np.median(Euclidean_Tip)))

ImageID: ipP.   Euclidean Distance in pixels: 40.8166632639171
ImageID: ipP.   Euclidean Distance in pixels: 5.0990195135927845
ImageID: ipP.   Euclidean Distance in pixels: 15.231546211727817
ImageID: ipP.   Euclidean Distance in pixels: 9.486832980505138
ImageID: ipP.   Euclidean Distance in pixels: 7.0710678118654755
ImageID: ipP.   Euclidean Distance in pixels: 3.1622776601683795
ImageID: ipP.   Euclidean Distance in pixels: 1.0
ImageID: ipP.   Euclidean Distance in pixels: 6.0
ImageID: ipP.   Euclidean Distance in pixels: 3.605551275463989
ImageID: ipP.   Euclidean Distance in pixels: 1.4142135623730951
ImageID: ipP.   Euclidean Distance in pixels: 48.41487374764082
ImageID: ipP.   Euclidean Distance in pixels: 31.38470965295043
ImageID: ipP.   Euclidean Distance in pixels: 26.832815729997478
ImageID: ipP.   Euclidean Distance in pixels: 4.123105625617661
ImageID: ipP.   Euclidean Distance in pixels: 5.385164807134504
ImageID: ipP.   Euclidean Distance in pixels: 126.6570171763096

ImageID: ipP.   Euclidean Distance in pixels: 13.152946437965905
ImageID: ipP.   Euclidean Distance in pixels: 51.62363799656123
ImageID: ipP.   Euclidean Distance in pixels: 8.06225774829855

-----------------------Final Mean Euclidean Distance (mED)---------------------------

Mean Euclidean Distance (mED) in pixels: 19.495004035747453
std. Euclidean Distance (ED): 26.287311382568443  median Euclidean Distance (ED): 6.708203932499369
